- 단계구분도: 경기도내 인구소멸비율

In [45]:
import json, folium
import numpy as np
import pandas as pd

- 경기도 시군구별 인구소멸비율 구하기

In [46]:
pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수','60~69세','70~79세','80~89세','90~99세','100세 이상','20~29세 (여)','30~39세 (여)']]
pop['인구소멸비율'] = (pop.iloc[:, -2:].sum(axis=1) / pop.iloc[:, 3:8].sum(axis=1)).round(3)
pop = pop[['행정구역구분명','행정구역명','총 인구수','인구소멸비율']]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
0,도,경기도,13589432,0.571
1,시군,경기도 가평군,62197,0.198
2,읍면동,경기도 가평군 가평읍,19555,0.257
3,읍면동,경기도 가평군 북면,3815,0.090
4,읍면동,경기도 가평군 상면,5669,0.114


In [47]:
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.198
8,시군,경기도 고양시,1075202,0.570
9,구,경기도 고양시 덕양구,487874,0.555
31,구,경기도 고양시 일산동구,296590,0.605
44,구,경기도 고양시 일산서구,290738,0.562


In [50]:
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
1,시군,경기도 가평군,62197,0.198
9,구,경기도 고양시 덕양구,487874,0.555
31,구,경기도 고양시 일산동구,296590,0.605
44,구,경기도 고양시 일산서구,290738,0.562
56,시군,경기도 과천시,77775,0.624


- geo data 읽기

In [51]:
filename = 'data/경기도_geo_simple.json'
geo_data = json.load(open(filename, encoding='utf-8'))

- 인구 데이터와 geo 데이터간 인덱스 일치 시키기

In [52]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율,시군구
1,시군,경기도 가평군,62197,0.198,가평군
9,구,경기도 고양시 덕양구,487874,0.555,고양시덕양구
31,구,경기도 고양시 일산동구,296590,0.605,고양시일산동구
44,구,경기도 고양시 일산서구,290738,0.562,고양시일산서구
56,시군,경기도 과천시,77775,0.624,과천시


In [53]:
pop.set_index('시군구', inplace=True)
pop.head()

,행정구역구분명,행정구역명,총 인구수,인구소멸비율
시군구,,,,
가평군,시군,경기도 가평군,62197,0.198
고양시덕양구,구,경기도 고양시 덕양구,487874,0.555
고양시일산동구,구,경기도 고양시 일산동구,296590,0.605
고양시일산서구,구,경기도 고양시 일산서구,290738,0.562
과천시,시군,경기도 과천시,77775,0.624


- 단계구분도

In [54]:
map = folium.Map([37.6, 127.1], zoom_start=8, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_data,
    data=pop.인구소멸비율,                    # 단계구분도로 보여줄 데이터
    columns=[pop.index, pop.인구소멸비율],    # 데이터프레임에서 추출할 항목
    fill_color='YlOrRd',                      # Colormap
    key_on='properties.name'                  # 지도에서 조인할 항목
).add_to(map)
map

- 시군구 이름 쓰기

In [55]:
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

In [56]:
sigungu_dict = get_text_location(geo_data)
for i, sigungu in enumerate(sigungu_dict.keys()):
    if i == 5:
        break
    print(f'{sigungu}: {sigungu_dict[sigungu]}')

양평군: [37.524478319941124, 127.57566999921767]
가평군: [37.80580217057866, 127.46168954149552]
연천군: [38.089173518920326, 126.98309880050942]
여주시: [37.30448255424536, 127.59166773209084]
포천시: [37.96989171146047, 127.23835163385141]


In [57]:
for sigungu_name in pop.index:
    folium.map.Marker(
        location=sigungu_dict[sigungu_name],
        icon = folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
            html=f'<div style="font-size: 8pt">{sigungu_name}</div>'
        )
    ).add_to(map)
map